### Annotation from predicted structures 

After preparing files for structures prediction of the cluster representatives on the HPC for running FoldSeek against PDB, AF50M and Phold db, moved on to annotation of structures

- PDB database version: latest update from 2024-11-29 - update on staging to more recent
- AlphaFold database version: latest Alphafold v2.3.2 march 2023 - same as current one available on staging
-   Phold database version: v0.2.0 2024-07-13

Steps followed:  
1. Select the set of the 1495 cluster rep - proteins
2. Prepare necessary files for predicting protein structures of these proteins on HPC.
3. Predict protein structures of these proteins. 
4. Prepare necessary files for analysing protein structures of these proteins on HPC.
5. Investigate the new function based on structures of each cluster.


Taking all template files:  submit_vibfold.py, VIBFold.py, VIBFold_adapted_functions.py and submit.sh stored in ../templates : template job inputs and scripts to be modified:

in folder structure_prediction:
in each folder batch_XX with XX a batch number between 0 and 20:
in folder fastas:
    - PROTEIN.fasta: fasta file for every hypothetical protein selected for structure prediction (so a cluster representative)
- submit_vibfold_PROTEIN.py: python script to submit VIBFold job for specific protein
- submit.sh, VIBFold.py, VIBFold_adapted_functions.py: python/bash scripts to submit and run VIBFold jobs
- VIBFOLD_PROTEIN.eXXXXX and VIBFOLD_PROTEIN.oXXXXX: error and output files for VIBFold jobs (generated after run on HPC)
- in folder results (generated after run on HPC):
in folder PROTEIN:
5 unrelaxed .pdb protein structures (PROTEIN_unrelaxed_rank_X_model_X_ptmx_1.pdb) and 1 relaxed .pdb protein structure (PROTEIN_relaxed.pdb): AlphaFold predicted structures3 .png files: 2 for PAE (one for all models - PROTEIN_PAE.png, one for the best model - best_PAE.png) and 1 for pLDDT/coverage (PROTEIN_coverage_lddt.png): quality metrics AlphaFold predicted structures depicted visually1 PROTEIN_info.log file1 PROTEIN_msa_ids.txt: .txt file containing all sequence identifiers of the sequences used in the MSA. Every line is a new identifier.query.fasta: input sequence used for AlphaFold run1 PROTEIN_rank_1_model_X_ptmx_1.json: json file with PAE scores of the best AlphaFold prediction



In [ ]:
# imports
import csv
import json
import os
import requests
import shutil
import sys
import pandas as pd
import Bio
from Bio.PDB import *
from json.decoder import JSONDecodeError
from more_itertools import collapse
import more_itertools as mit
from json import JSONDecodeError
import re
from more_itertools import chunked

In [ ]:
# fetching sequences of clust representatives
merged_df = pd.read_csv('for_protein_fasta.tsv', sep='\t')
os.makedirs('fastas', exist_ok=True)
for index, row in merged_df.iterrows():
    fasta_filename = f"fastas/{row['id']}.fasta"
    with open(fasta_filename, 'w') as fasta_file:
        fasta_file.write(f">{row['id']}\n{row['sequence']}\n")

print("FASTA files have been created in the 'fastas' directory.")

In [ ]:
# clear reference to different directories
pipeline_search_dir = os.getcwd()
master_dir = os.path.abspath(os.path.join(pipeline_search_dir, os.pardir, os.pardir))

In [ ]:
os.mkdir(os.path.join(pipeline_search_dir, "c_structure_annotation", "structure_prediction")) 

In [ ]:
# function to adapt job scripts starting from an adapted version of Jasper Zuallaert's submit_vibfold.py
def adapt_vibfold_jobscript(template, infile, outfile, length):
    # adapt runtime job based on length protein
    if length < 301:
        time = "1:30:00"
    elif length > 300 and length < 501:
        time = "3:30:00"
    else:
        time = "5:00:00"
    #read in data
    with open(template, "r") as file:
        filedata = file.read()
    #change script
    filedata = filedata.replace("inputfasta", "fastas/{0}".format(str(infile)))
    filedata = filedata.replace("walltime=48:00:00", "walltime={0}".format(time))
    #write new job script
    with open(outfile, "w") as file:
        file.write(filedata)

In [ ]:
# Preparing files to submit to HPC
template = os.path.join(master_dir, "templates", "submit_vibfold.py")

data_path = 'for_protein_fasta.tsv'
data = pd.read_csv(data_path, sep='\t')
batches = chunked(data['id'].tolist(), 500)

for index_batch, batch in enumerate(batches):
    batch_dir = os.path.join(pipeline_search_dir, "c_structure_annotation", "structure_prediction", f"batch_{index_batch}")
    fasta_dir = os.path.join(batch_dir, "fastas")
    results_dir = os.path.join(batch_dir, "results")

    os.makedirs(fasta_dir, exist_ok=True)
    os.makedirs(results_dir, exist_ok=True)

    shutil.copyfile(os.path.join(master_dir, "templates", "VIBFold.py"), os.path.join(batch_dir, "VIBFold.py"))
    shutil.copyfile(os.path.join(master_dir, "templates", "VIBFold_adapted_functions.py"), os.path.join(batch_dir, "VIBFold_adapted_functions.py"))
    shutil.copyfile(os.path.join(master_dir, "templates", "submit.sh"), os.path.join(batch_dir, "submit.sh"))

    batch_data = data[data['id'].isin(batch)]

    for index_prot, row in batch_data.iterrows():
        prot_id = row['id']  
        sequence = row['sequence']
        length = row.get('length', 0)  

        fasta_file_path = os.path.join(fasta_dir, f"{prot_id}.fasta")
        with open(fasta_file_path, "w") as fasta_file:
            fasta_file.write(f">{prot_id}\n{sequence}\n")

        # Prepare and write the job script
        submit_path = os.path.join(batch_dir, f"submit_vibfold_{prot_id}.py")
        template = os.path.join(master_dir, "templates", "submit_vibfold.py")
        adapt_vibfold_jobscript(template, f"{prot_id}.fasta", submit_path, length)

        with open(os.path.join(batch_dir, "submit.sh"), "a+") as submit_file:
            submit_file_name = f"submit_vibfold_{prot_id}.py"
            submit_file.write(f"python {submit_file_name} \n")



After running FoldSeek, must prepare files for analysis and annotation

In [ ]:
data_path = 'for_protein_fasta.tsv'
data = pd.read_csv(data_path, sep='\t')  
filtered_clustered_reps_int = data['id'].tolist()

In [ ]:
base_path = "c_structure_annotation/structure_prediction"

directory_names = []

for batch in ["batch_0", "batch_1", "batch_2"]:
    results_path = os.path.join(base_path, batch, "results")
    
    # Check each directory within the results folder
    for entry in os.listdir(results_path):
        full_path = os.path.join(results_path, entry)
        if os.path.isdir(full_path):
            directory_names.append(entry)

#print(directory_names)

In [ ]:
proteins_structure_success = directory_names.copy()

In [ ]:
#copying of relaxed structures
comparison_dir = os.path.join(pipeline_search_dir, "c_structure_annotation", "structure_comparison")

os.makedirs(comparison_dir, exist_ok=True)

for protein in directory_names:
    batch_path = os.path.join(pipeline_search_dir, "c_structure_annotation", "structure_prediction", "batch_0" , "results")
    relaxed_path = os.path.join(batch_path, protein, f"{protein}_relaxed.pdb")
    if os.path.exists(relaxed_path):
        shutil.copyfile(relaxed_path, os.path.join(comparison_dir, f"{protein}_relaxed.pdb"))

In [ ]:
# Seperating into batches
def batched(iterable, n):
    """Helper function to divide an iterable into batches of size n."""
    length = len(iterable)
    for ndx in range(0, length, n):
        yield iterable[ndx:min(ndx + n, length)]

base_dir = "c_structure_annotation/structure_comparison"
files = [f for f in os.listdir(base_dir) if os.path.isfile(os.path.join(base_dir, f))]
for index_batch, batch in enumerate(batched(files, 300)):
    batch_dir = os.path.join(base_dir, f"batch_{index_batch}")
    os.makedirs(batch_dir, exist_ok=True)

    for filename in batch:
        source_path = os.path.join(base_dir, filename)
        destination_path = os.path.join(batch_dir, filename)
        shutil.move(source_path, destination_path)

In [1]:
!pip install more-itertools 

Defaulting to user installation because normal site-packages is not writeable
    matplotlib-inline (<0.2.0appnope,>=0.1.0) ; platform_system == "Darwin"
                      ~~~~~~~~^

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install biopython 


Defaulting to user installation because normal site-packages is not writeable
    matplotlib-inline (<0.2.0appnope,>=0.1.0) ; platform_system == "Darwin"
                      ~~~~~~~~^

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!conda list --explicit 

# This file may be used to create an environment using:
# $ conda create --name <env> --file <this file>
# platform: linux-64
# created-by: conda 24.11.0
@EXPLICIT
https://repo.anaconda.com/pkgs/main/linux-64/_libgcc_mutex-0.1-main.conda
https://repo.anaconda.com/pkgs/main/linux-64/ca-certificates-2024.11.26-h06a4308_0.conda
https://repo.anaconda.com/pkgs/main/linux-64/ld_impl_linux-64-2.38-h1181459_1.conda
https://repo.anaconda.com/pkgs/main/linux-64/libstdcxx-ng-11.2.0-h1234567_1.conda
https://repo.anaconda.com/pkgs/main/noarch/pybind11-abi-5-hd3eb1b0_0.conda
https://repo.anaconda.com/pkgs/main/noarch/tzdata-2024a-h04d1e81_0.conda
https://repo.anaconda.com/pkgs/main/linux-64/libgomp-11.2.0-h1234567_1.conda
https://repo.anaconda.com/pkgs/main/linux-64/_openmp_mutex-5.1-1_gnu.conda
https://repo.anaconda.com/pkgs/main/linux-64/libgcc-ng-11.2.0-h1234567_1.conda
https://repo.anaconda.com/pkgs/main/linux-64/bzip2-1.0.8-h5eee18b_6.conda
https://repo.anaconda.com/pkgs/main/linux-64/c-ares-1.

In [5]:
# Settings for requests
sess = requests.Session()
adapter = requests.adapters.HTTPAdapter(max_retries = 10)
sess.mount("https://", adapter)

In [6]:
# Clear reference to different directories
pipeline_search_dir = os.getcwd()
master_dir = os.path.abspath(os.path.join(pipeline_search_dir, os.pardir, os.pardir))

Processing the FoldSeek result files

Using the created scripts, protein structures were compared to the PDB, Phold and AlphaFold database on the HPC. Results were downloaded, and all files created in the process are stored in  subdirectory 'structure_comparison'. While the comparison of our structure has been done, the output files are not very interpretable: matched proteins are described based on UniProt ID or PDB ID or Phrog ID.

Hence, we have to process the output to link protein names to these identifiers. In addition, we will also apply some quality filters to our hits. Briefly, we will filter based on:
- probability of our protein and its match to belong to the same SCOP class: > 50%
- quality of the alignment: alignment lDDT > 50% (based on visual inspection of alignments)
- likelihood of the hit: E-value cutoff E-3 (FoldSeek default)
- quality of the aligned region of input protein: average plDDT of aligned region > 70 (standard AlphaFold cut-off)

In order to do so, we write functions to extract these features, and to match the PDB/UniProt/Phrog information to protein names. 


In [2]:
base_path = "c_structure_annotation/structure_prediction"
directory_names = []

# Loop through each batch folder
for batch in ["batch_0", "batch_1", "batch_2"]:
    results_path = os.path.join(base_path, batch, "results")
    for entry in os.listdir(results_path):
        full_path = os.path.join(results_path, entry)
        if os.path.isdir(full_path):
            directory_names.append(entry)

# check cluster rep which structure has been successfully predicted
#print(directory_names)

bugs for following proteins: so remove
CAR312481
Station158_DCM_ALL_assembly_NODE_1847_length_35185_cov_4285255_36
biochar_58_26
biochar_267_6
biochar_464_13
biochar_638_45
biochar_884_13
biochar_2218_16
biochar_2253_1
biochar_2974_7
biochar_3169_9
biochar_3173_15
biochar_3481_12
biochar_3496_7
biochar_3567_15
biochar_3928_1
biochar_4120_8
biochar_4122_1
biochar_4254_1
biochar_4411_4
biochar_4538_2
biochar_4592_12
biochar_4597_10
biochar_4628_1
biochar_4628_4
biochar_4928_3
biochar_4936_1
biochar_4991_12
biochar_5172_12
biochar_5399_2
biochar_5439_1
biochar_5571_1
biochar_5862_1
biochar_5897_23
biochar_5984_13
biochar_6075_12

In [9]:
# creating a list of all protein IDs for which structure prediction succeeded 
    # this is all the proteins for which structure prediction was started
proteins_structure_success = directory_names.copy()
    # except the NaN cluster representative and the buggy ones
#proteins_structure_success.remove('CAR312481')

bug_ids = ['CAR312481','Station158_DCM_ALL_assembly_NODE_1847_length_35185_cov_4285255_36',
'biochar_58_26','biochar_267_6','biochar_464_13','biochar_638_45','biochar_884_13',
'biochar_2218_16','biochar_2253_1','biochar_2974_7','biochar_3169_9','biochar_3173_15','biochar_3481_12',
'biochar_3496_7','biochar_3567_15','biochar_3928_1','biochar_4120_8','biochar_4122_1','biochar_4254_1',
'biochar_4411_4','biochar_4538_2','biochar_4592_12','biochar_4597_10','biochar_4628_1','biochar_4628_4',
'biochar_4928_3','biochar_4936_1','biochar_4991_12','biochar_5172_12', 'biochar_5399_2', 'biochar_5439_1',
'biochar_5571_1','biochar_5862_1','biochar_5897_23','biochar_5984_13', 'biochar_6075_12']

for id in bug_ids:
    proteins_structure_success.remove(id)
# creating a dictionary storing whether relaxed or best structure should be used
    # list of the IDs (based on output checks)

#now add list where relaxation failed and using best is better
# creating a dictionary storing whether relaxed or best structure should be used
    # list of the IDs (based on output checks)
relaxation_failed_ids = ['']
    
    # creating dictionary
relax_best_dict = {}
for key in proteins_structure_success:
    if key in proteins_structure_success:
        relax_best_dict[key] = "relax"
 

In [ ]:
# Using `chunked` to create chunks of size 500 from `filtered_clustered_reps_int`
for index_chunk, chunk in enumerate(mit.chunked(proteins_structure_success, 500)):
    batch_path = os.path.join(pipeline_search_dir, "c_structure_annotation", "structure_prediction", f"batch_{index_chunk}")
    
    # Ensure the batch_path directory exists
    if not os.path.exists(batch_path):
        os.makedirs(batch_path)
    
    # Iterate through each protein in the chunk
    for protein in chunk:
        # Path for proteins with relaxed structure
        relaxed_path = os.path.join(batch_path, "results", f"{protein}", f"{protein}_relaxed.pdb")
        
        # Check if the relaxed structure file exists before copying
        if os.path.exists(relaxed_path):
            destination_path = os.path.join(pipeline_search_dir, "c_structure_annotation", "structure_comp", f"{protein}_relaxed.pdb")
            
            # Ensure the destination directory exists
            destination_dir = os.path.dirname(destination_path)
            if not os.path.exists(destination_dir):
                os.makedirs(destination_dir)
            
            shutil.copyfile(relaxed_path, destination_path)
        else:
            print(f" {protein}")


As there were some issues with file not found when running the code the first time, added checks to ensure every relaxed file is successfully added and reran problematic ones

In [7]:
#filtering foldseek results
# define a PDB parser
p = PDBParser()

In [8]:
# function for calculating average pLDDT of aligned region on results
def get_pLDDT(structure, start, stop):
    for model in structure:
        for chain in model:
            count_residue = 0
            pLDDT_chain = 0
            for residue in chain:
                if residue.id[1] in range(start,stop+1):
                    count_residue += 1
                    count_atom = 0
                    pLDDT_atom = 0
                    for atom in residue:
                        count_atom += 1
                        pLDDT_atom += atom.get_bfactor()
                    pLDDT_res = round(pLDDT_atom/count_atom,2)
                    pLDDT_chain += pLDDT_res
            pLDDT_range = round(pLDDT_chain/count_residue,2)
    return pLDDT_range 

in structure comp, we have all the proteins for which there was successful prediction (relaxed file exists). we now want to create a few folders/batches to seperate these files for easier parsing

In [ ]:
SOURCE_DIR = "c_structure_annotation/structure_comp"  
MAX_FILES_PER_FOLDER = 300

# Get all files in the directory
files = [f for f in os.listdir(SOURCE_DIR) if os.path.isfile(os.path.join(SOURCE_DIR, f))]

# Create and move files into numbered folders in chunks of 300
for i in range(0, len(files), MAX_FILES_PER_FOLDER):
    folder_name = os.path.join(SOURCE_DIR, f"batch_{i // MAX_FILES_PER_FOLDER + 1}")
    os.makedirs(folder_name, exist_ok=True)
    
    for file in files[i:i + MAX_FILES_PER_FOLDER]:
        shutil.move(os.path.join(SOURCE_DIR, file), os.path.join(folder_name, file))

In [ ]:
def batched(iterable, n):
    """Helper function to divide an iterable into batches of size n."""
    length = len(iterable)
    for ndx in range(0, length, n):
        yield iterable[ndx:min(ndx + n, length)]

pipeline_search_dir = "c_structure_annotation/structure_comp"
master_dir = "c_structure_annotation"  

batch_dirs = [name for name in os.listdir(pipeline_search_dir) if os.path.isdir(os.path.join(pipeline_search_dir, name)) and name.startswith("batch_")]

for batch_dir_name in batch_dirs:
    batch_dir = os.path.join(pipeline_search_dir, batch_dir_name)

    shutil.copyfile(os.path.join(master_dir, "data_foldseek_5.csv"),
                    os.path.join(batch_dir, "data_foldseek_5.csv"))

    files = [file for file in os.listdir(batch_dir) if os.path.isfile(os.path.join(batch_dir, file)) and file != "data_foldseek_5.csv"]
    with open(os.path.join(batch_dir, "data_foldseek_5.csv"), "a+") as data_file:
        for sub_batch in list(batched(files, 5)):
            line = ",".join([f"{filename},{filename.replace('.pdb', '_aln_pdb.txt')},{filename.replace('.pdb', '_aln_af50m.txt')}, {filename.replace('.pdb', '_phold.txt')}" for filename in sub_batch])
            data_file.write(line + "\n")
            

    shutil.copyfile(os.path.join(master_dir, "script_foldseek_5.slurm"),
                    os.path.join(batch_dir, "script_foldseek_5.slurm"))

print("Setup of job scripts and data files for each batch completed.")

In [ ]:
SOURCE_DIR = "c_structure_annotation/structure_comp" 

def batch(iterable, size):
    """Yield successive n-sized chunks from iterable."""
    iterator = iter(iterable)
    while chunk := list(islice(iterator, size)):
        yield chunk

def get_pLDDT(structure, start, stop):
    for model in structure:
        for chain in model:
            count_residue = 0
            pLDDT_chain = 0
            for residue in chain:
                if residue.id[1] in range(start,stop+1):
                    count_residue += 1
                    count_atom = 0
                    pLDDT_atom = 0
                    for atom in residue:
                        count_atom += 1
                        pLDDT_atom += atom.get_bfactor()
                    pLDDT_res = round(pLDDT_atom/count_atom,2)
                    pLDDT_chain += pLDDT_res
            pLDDT_range = round(pLDDT_chain/count_residue,2)
    return pLDDT_range 

# Get all existing batch folders
batch_folders = sorted([f for f in os.listdir(SOURCE_DIR) if os.path.isdir(os.path.join(SOURCE_DIR, f)) and f.startswith("batch_")])

for batch_folder in batch_folders:
    batch_dir = os.path.join(SOURCE_DIR, batch_folder)
    files = [f for f in os.listdir(batch_dir) if os.path.isfile(os.path.join(batch_dir, f))]

    for file in files:
        protein = os.path.splitext(file)[0]  
        structure = os.path.join(batch_dir, f"{protein}_relaxed.pdb")  
        

        # Process FoldSeek results against AF50m database
        af50m_path = os.path.join(batch_dir, f"{protein}_aln_af50m.txt")
        # filtering hits against AF50m database
            # reading in the data
        if os.path.exists(af50m_path):
            data_af50m = pd.read_table(af50m_path, header=None, names=["query","target","fident","alnlen","mismatch","gapopen","qstart","qend","tstart","tend","evalue","bits","prob","lddt","lddtfull"])
            
            # adding AlphaFold quality metrics - pLDDT of aligned region query

            # Add pLDDT column
            for i in range(len(data_af50m)):
                data_af50m.at[i, "pLDDT_qAln"] = get_pLDDT(structure, data_af50m.at[i, "qstart"], data_af50m.at[i, "qend"])

            # Apply filtering
            data_af50m_filtered = data_af50m[(data_af50m["prob"] > 0.5) & (data_af50m["evalue"] < 1e-3) & (data_af50m["lddt"] >= 0.5) & (data_af50m["pLDDT_qAln"] >= 0.7)]
            data_af50m_filtered = data_af50m_filtered.reset_index(drop=True)
            data_af50m_filtered.to_csv(os.path.join(batch_dir, f"{protein}_foldseek_af50m_filtered.csv"), index=False)

        # Process FoldSeek results against PDB database
        pdb_path = os.path.join(batch_dir, f"{protein}_aln_pdb.txt")
        if os.path.exists(pdb_path):
            data_pdb = pd.read_table(pdb_path, header=None, names=["query","target","fident","alnlen","mismatch","gapopen","qstart","qend","tstart","tend","evalue","bits","prob","lddt","lddtfull"])
            
            # Add pLDDT column
            for i in range(len(data_pdb)):
                data_pdb.at[i, "pLDDT_qAln"] = get_pLDDT(structure, data_pdb.at[i, "qstart"], data_pdb.at[i, "qend"])

            # Apply filtering
            data_pdb_filtered = data_pdb[(data_pdb["prob"] > 0.5) & (data_pdb["evalue"] < 1e-3) & (data_pdb["lddt"] >= 0.5) & (data_pdb["pLDDT_qAln"] >= 0.7)]
            data_pdb_filtered = data_pdb_filtered.reset_index(drop=True)

            # Save filtered results
            data_pdb_filtered.to_csv(os.path.join(batch_dir, f"{protein}_foldseek_pdb_filtered.csv"), index=False)

In [20]:
for index_batch, batch in enumerate(list(batched(relax_best_dict.keys(),300))):
    batch_dir = os.path.join(pipeline_search_dir, "c_structure_annotation", "structure_comparison", f"batch_{index_batch}")  
    for protein in batch:
        # get protein structure from AlphaFold
        if relax_best_dict.get(protein) == "relax":
            structure = p.get_structure(protein, os.path.join(batch_dir, "{0}_relaxed.pdb".format(protein)))
        if relax_best_dict.get(protein) == "best":
            structure = p.get_structure(protein, os.path.join(batch_dir, "{0}_best.pdb".format(protein)))
        # filtering hits against AF50m database
            # reading in the data
        data_af50m = pd.read_table(os.path.join(batch_dir, "{0}_aln_af50m.txt".format(protein)), header=None, names=["query","target","fident","alnlen","mismatch","gapopen","qstart","qend","tstart","tend","evalue","bits","prob","lddt","lddtfull"])
            # adding AlphaFold quality metrics - pLDDT of aligned region query
        for i in range(0,len(data_af50m)):
            data_af50m.at[i,"pLDDT_qAln"] = get_pLDDT(structure, data_af50m.at[i,"qstart"],data_af50m.at[i,"qend"])
            # filtering based on FoldSeek parameters & AF model accuracy
        data_af50m_filtered = data_af50m
        if len(data_af50m_filtered) != 0:
            data_af50m_filtered = data_af50m_filtered[(data_af50m_filtered["prob"] > 0.5) & (data_af50m_filtered["evalue"] < 1e-3) & (data_af50m_filtered["lddt"] >= 0.5) & (data_af50m_filtered["pLDDT_qAln"] >= 0.7)]
        data_af50m_filtered = data_af50m_filtered.reset_index()
            # store filtered FoldSeek results
        data_af50m_filtered.to_csv(os.path.join(batch_dir, f"{protein}_foldseek_af50m_filtered.csv"), index=False)     
        # filtering hits against PDB database
            # reading in the data
        data_pdb = pd.read_table(os.path.join(batch_dir, "{0}_aln_pdb.txt".format(protein)), header=None, names=["query","target","fident","alnlen","mismatch","gapopen","qstart","qend","tstart","tend","evalue","bits","prob","lddt","lddtfull"])
            # adding AlphaFold quality metrics - pLDDT of aligned region query
        for i in range(0,len(data_pdb)):
            data_pdb.at[i,"pLDDT_qAln"] = get_pLDDT(structure, data_pdb.at[i,"qstart"],data_pdb.at[i,"qend"])
            # filtering based on FoldSeek parameters & AF model accuracy
        data_pdb_filtered = data_pdb
        if len(data_pdb_filtered) != 0:
            data_pdb_filtered = data_pdb_filtered[(data_pdb_filtered["prob"] > 0.5) & (data_pdb_filtered["evalue"] < 1e-3) & (data_pdb_filtered["lddt"] >= 0.5) & (data_pdb_filtered["pLDDT_qAln"] >= 0.7)]
        data_pdb_filtered = data_pdb_filtered.reset_index()
            # store filtered FoldSeek results
        data_pdb_filtered.to_csv(os.path.join(batch_dir, f"{protein}_foldseek_pdb_filtered.csv"), index=False) 

In [23]:
def get_pLDDT(structure, start, stop):
    for model in structure:
        for chain in model:
            count_residue = 0
            pLDDT_chain = 0
            for residue in chain:
                if residue.id[1] in range(start,stop+1):
                    count_residue += 1
                    count_atom = 0
                    pLDDT_atom = 0
                    for atom in residue:
                        count_atom += 1
                        pLDDT_atom += atom.get_bfactor()
                    pLDDT_res = round(pLDDT_atom/count_atom,2)
                    pLDDT_chain += pLDDT_res
            pLDDT_range = round(pLDDT_chain/count_residue,2)
    return pLDDT_range 

# Define parent directory where batches are stored
pipeline_search_dir = "c_structure_annotation/structure_comp"  

# Loop through batch directories
for index_batch in range(1, 6):  # Adjust range if needed
    batch_dir = os.path.join(pipeline_search_dir, f"batch_{index_batch}")

    # Loop through protein files in the batch directory
    for filename in os.listdir(batch_dir):
        if filename.endswith("_relaxed.pdb"):  # Ensure processing only PDB files
            protein = filename.replace("_relaxed.pdb", "")  # Extract protein name

            # Load protein structure (since all are "relax")
            structure = p.get_structure(protein, os.path.join(batch_dir, filename))

            # Process FoldSeek results for AF50m database
            af50m_file = os.path.join(batch_dir, f"{protein}_aln_af50m.txt")
            if os.path.exists(af50m_file):
                data_af50m = pd.read_table(af50m_file, header=None, names=[
                    "query", "target", "fident", "alnlen", "mismatch", "gapopen", "qstart", "qend",
                    "tstart", "tend", "evalue", "bits", "prob", "lddt", "lddtfull"
                ])
                
                # Add AlphaFold quality metrics
                for i in range(len(data_af50m)):
                    data_af50m.at[i, "pLDDT_qAln"] = get_pLDDT(structure, data_af50m.at[i, "qstart"], data_af50m.at[i, "qend"])
                
                # Apply filtering
                data_af50m_filtered = data_af50m[(data_af50m["prob"] > 0.5) & (data_af50m["evalue"] < 1e-3) &
                                                 (data_af50m["lddt"] >= 0.5) & (data_af50m["pLDDT_qAln"] >= 0.7)]
                
                # Save filtered results
                data_af50m_filtered.to_csv(os.path.join(batch_dir, f"{protein}_foldseek_af50m_filtered.csv"), index=False)

            # Process FoldSeek results for PDB database
            pdb_file = os.path.join(batch_dir, f"{protein}_aln_pdb.txt")
            if os.path.exists(pdb_file):
                data_pdb = pd.read_table(pdb_file, header=None, names=[
                    "query", "target", "fident", "alnlen", "mismatch", "gapopen", "qstart", "qend",
                    "tstart", "tend", "evalue", "bits", "prob", "lddt", "lddtfull"
                ])
                
                # Add AlphaFold quality metrics
                for i in range(len(data_pdb)):
                    data_pdb.at[i, "pLDDT_qAln"] = get_pLDDT(structure, data_pdb.at[i, "qstart"], data_pdb.at[i, "qend"])
                
                # Apply filtering
                data_pdb_filtered = data_pdb[(data_pdb["prob"] > 0.5) & (data_pdb["evalue"] < 1e-3) &
                                             (data_pdb["lddt"] >= 0.5) & (data_pdb["pLDDT_qAln"] >= 0.7)]
                
                # Save filtered results
                data_pdb_filtered.to_csv(os.path.join(batch_dir, f"{protein}_foldseek_pdb_filtered.csv"), index=False)

In [ ]:
pipeline_search_dir = "c_structure_annotation"  
# Iterate over batch folders from batch_1 to batch_5
for index_batch in range(1, 6):  
    batch_dir = os.path.join(pipeline_search_dir, "structure_comp", f"batch_{index_batch}")

    # Check all proteins in the batch directory
    for filename in os.listdir(batch_dir):
        if filename.endswith("_relaxed.pdb"):  
            protein = filename.replace("_relaxed.pdb", "") 

            alnfile_af50m = os.path.join(batch_dir, f"{protein}_relaxed_aln_af50m.txt")
            alnfile_pdb = os.path.join(batch_dir, f"{protein}_relaxed_aln_pdb.txt")
            alnfile_phold = os.path.join(batch_dir, f"{protein}_relaxed_phold.txt")

            # Check if all files exist
            if not (os.path.isfile(alnfile_af50m) and os.path.isfile(alnfile_pdb)):
                #print(f"Issue finding FoldSeek output files for protein {protein} in batch_{index_batch}.")

5. Investigate function based on structures

In [8]:
# define a PDB parser
p = PDBParser()

In [9]:
# function for calculating average pLDDT of aligned region on FoldSeek results
def get_pLDDT(structure, start, stop):
    for model in structure:
        for chain in model:
            count_residue = 0
            pLDDT_chain = 0
            for residue in chain:
                if residue.id[1] in range(start,stop+1):
                    count_residue += 1
                    count_atom = 0
                    pLDDT_atom = 0
                    for atom in residue:
                        count_atom += 1
                        pLDDT_atom += atom.get_bfactor()
                    pLDDT_res = round(pLDDT_atom/count_atom,2)
                    pLDDT_chain += pLDDT_res
            pLDDT_range = round(pLDDT_chain/count_residue,2)
    return pLDDT_range

In [44]:
pipeline_search_dir = "c_structure_annotation/structure_comp"  

for index_batch in range(1, 6):  
    batch_dir = os.path.join(pipeline_search_dir, f"batch_{index_batch}")

    for filename in os.listdir(batch_dir):
        if filename.endswith("_relaxed.pdb"):  
            protein = filename.replace("_relaxed.pdb", "")  
            structure = p.get_structure(protein, os.path.join(batch_dir, "{0}_relaxed.pdb".format(protein)))

            # Process FoldSeek results for AF50m database
            data_af50m = pd.read_table(os.path.join(batch_dir, "{0}_relaxed_aln_af50m.txt".format(protein)), header=None, names=["query","target","fident","alnlen","mismatch","gapopen","qstart","qend","tstart","tend","evalue","bits","prob","lddt","lddtfull"])

        for i in range(0,len(data_af50m)):
            data_af50m.at[i,"pLDDT_qAln"] = get_pLDDT(structure, data_af50m.at[i,"qstart"],data_af50m.at[i,"qend"])
      
        data_af50m_filtered = data_af50m
        if len(data_af50m_filtered) != 0:
            data_af50m_filtered = data_af50m_filtered[(data_af50m_filtered["prob"] > 0.5) & (data_af50m_filtered["evalue"] < 1e-3) & (data_af50m_filtered["lddt"] >= 0.5) & (data_af50m_filtered["pLDDT_qAln"] >= 0.7)]
        data_af50m_filtered = data_af50m_filtered.reset_index()
       
        data_af50m_filtered.to_csv(os.path.join(batch_dir, f"{protein}_foldseek_af50m_filtered.csv"), index=False)     
     
            # Process FoldSeek results for PDB database
        
        data_pdb = pd.read_table(os.path.join(batch_dir, "{0}_relaxed_aln_pdb.txt".format(protein)), header=None, names=["query","target","fident","alnlen","mismatch","gapopen","qstart","qend","tstart","tend","evalue","bits","prob","lddt","lddtfull"])

        for i in range(0,len(data_pdb)):
            data_pdb.at[i,"pLDDT_qAln"] = get_pLDDT(structure, data_pdb.at[i,"qstart"],data_pdb.at[i,"qend"])
         
        data_pdb_filtered = data_pdb
       
        if len(data_pdb_filtered) != 0:
            data_pdb_filtered = data_pdb_filtered[(data_pdb_filtered["prob"] > 0.5) & (data_pdb_filtered["evalue"] < 1e-3) & (data_pdb_filtered["lddt"] >= 0.5) & (data_pdb_filtered["pLDDT_qAln"] >= 0.7)]
        data_pdb_filtered = data_pdb_filtered.reset_index()
                # store filtered FoldSeek results
        data_pdb_filtered.to_csv(os.path.join(batch_dir, f"{protein}_foldseek_pdb_filtered.csv"), index=False) 

In [ ]:
#now adding for treating phold - phrogs foldseek results
pipeline_search_dir = "c_structure_annotation/structure_comp"  

for index_batch in range(1, 6):  
    batch_dir = os.path.join(pipeline_search_dir, f"batch_{index_batch}")

    for filename in os.listdir(batch_dir):
        if filename.endswith("_relaxed.pdb"): 
            protein = filename.replace("_relaxed.pdb", "")  

            structure = p.get_structure(protein, os.path.join(batch_dir, "{0}_relaxed.pdb".format(protein)))

            # Process FoldSeek results for Phrogs database
            data_phold = pd.read_table(os.path.join(batch_dir, "{0}_relaxed_phold.txt".format(protein)), header=None, names=["query","target","fident","alnlen","mismatch","gapopen","qstart","qend","tstart","tend","evalue","bits","prob","lddt","lddtfull"])

        for i in range(0,len(data_phold)):
            data_phold.at[i,"pLDDT_qAln"] = get_pLDDT(structure, data_phold.at[i,"qstart"],data_phold.at[i,"qend"])
      
        data_phold_filtered = data_phold
        if len(data_phold_filtered) != 0:
            ddata_phold_filtered = data_phold_filtered[(data_phold_filtered["prob"] > 0.5) & (data_phold_filtered["evalue"] < 1e-3) & (data_phold_filtered["lddt"] >= 0.5) & (data_phold_filtered["pLDDT_qAln"] >= 0.7)]
        data_phold_filtered = data_phold_filtered.reset_index()
       
        data_phold_filtered.to_csv(os.path.join(batch_dir, f"{protein}_foldseek_phold_filtered.csv"), index=False)     

In [13]:
pipeline_search_dir = "c_structure_annotation"  

for index_batch in range(1, 6):  
    batch_dir = os.path.join(pipeline_search_dir, "structure_comp", f"batch_{index_batch}")

    # Check all proteins in the batches
    for filename in os.listdir(batch_dir):
        if filename.endswith("_relaxed.pdb"):  
            protein = filename.replace("_relaxed.pdb", "")  

            csv_af50m = os.path.join(batch_dir, f"{protein}_foldseek_af50m_filtered.csv")
            csv_pdb = os.path.join(batch_dir, f"{protein}_foldseek_pdb_filtered.csv")
            csv_phold = os.path.join(batch_dir, f"{protein}_foldseek_phold_filtered.csv")

            # Check if all files exist
            if not (os.path.isfile(csv_af50m) and os.path.isfile(csv_pdb)) and (os.path.isfile(csv_phold)) :
                print(f"Issue finding FoldSeek filtered output files for protein {protein} in batch_{index_batch}.")



No error message -  all csv files created successfully for each protein in every batch, now moving on to analysing results

#### Analysing outputs - Extracting the UniProt IDs from the filtered FoldSeek output files
Next, we will extract the UniProt IDs from the filtered FoldSeek results. For the searches against the AlphaFold database, this is quite straightforward (the UniProt ID can be extracted directly from the target column), but for the searched against the PDB database, this requires linking PDB entries to UniProt IDs. After extraction, the unique identifiers are stored in dictionaries, linking the UniProt ID to the protein for which a structure model of this UniProt ID was a hit. For the PDB entries, we also store the connection between UniProt ID and PDB ID with chain in a seperate dictionary. For reproducibility purposes, we will store these dictionaries and the list of obsolete PDB identifiers on the date of search to .csv files, as database updates could make this step of the pipeline non reproducible.First, we'll deal with the output files from the search against the AlphaFold database.

In [22]:
# function to extract UniProt ID from the FoldSeek target column
def get_UniProtID_from_target(target):
    uniprot_id = target.split('-')[1]
    return uniprot_id

In [11]:
# instantiate empty dictionary
dict_af50m_UniProtID_NCBIid = {}

In [16]:
# loop through batches, then through proteins

pipeline_search_dir = "c_structure_annotation/structure_comp"  
for index_batch in range(1, 6): 
    batch_dir = os.path.join(pipeline_search_dir, f"batch_{index_batch}")
    protein_files = [f for f in os.listdir(batch_dir) if f.endswith('_foldseek_af50m_filtered.csv')]
    
    for file_name in protein_files:
        protein = file_name.replace('_foldseek_af50m_filtered.csv', '')
   
        filtered_foldseek = pd.read_csv(os.path.join(batch_dir, f"{protein}_foldseek_af50m_filtered.csv"))
        if not filtered_foldseek.empty:
            for i in range(0,len(filtered_foldseek)):
                # extract UniProt ID
                uniprot_id = get_UniProtID_from_target(filtered_foldseek.at[i, "target"])
                # if UniProt ID already in dictionary: add NCBI ID to list of already matched NCBI IDs as value for the UniProt ID
                if uniprot_id in dict_af50m_UniProtID_NCBIid.keys():
                    prot_list = dict_af50m_UniProtID_NCBIid.get(uniprot_id)
                # if UniProt ID not yet in dictionary: create new key-value pair linking NCBI ID to UniProt ID
                if uniprot_id not in dict_af50m_UniProtID_NCBIid.keys():
                    prot_list = list()
                prot_list.append(protein)
                dict_af50m_UniProtID_NCBIid[uniprot_id] = list(set(prot_list))

In [17]:
  # store dict to file
    # here, this is not needed for reproducibility, but better to be consistent + saves time when rerunning
with open(os.path.join(pipeline_search_dir, "dict_af50m_UniProtID_NCBIid.csv"), "w", newline="") as fp:
    # create a writer object
    writer = csv.DictWriter(fp, fieldnames = dict_af50m_UniProtID_NCBIid.keys())
    writer.writeheader()
    writer.writerow(dict_af50m_UniProtID_NCBIid)

we have the alphafold results
now moving on to PDB results: output files against PDB dataset

In [18]:
# list of obsolete PDB IDs on date of search (Feb 8th, 2025)
obsolete_ids = Bio.PDB.PDBList.get_all_obsolete(Bio.PDB.PDBList())

In [19]:
# store list of obsolete PDB IDs to file
with open(os.path.join(pipeline_search_dir, "list_obsolete_PDB_ids.txt"), "w+") as file:
    for pdb_id in obsolete_ids:
        file.write(f"{pdb_id} \n")
file.close()

In [24]:
# necessary functions to ...

# from FoldSeek target column, extract PDB id and chain
def get_PDBid_chain_from_target(target):
    pdb_id = target.split('.')[0]
    chain = target.split('_')[-1]
    return pdb_id, chain

# check if PDB ID is obsolete
def check_PDBid_active(pdb_id, obsolete_list):
    if pdb_id.upper() in obsolete_list:
        return False
    else:
        return True

# from PDB ID, get all PDB polymer entities
def get_PDB_entities_from_entry(pdb_id):
    url = f"https://data.rcsb.org/rest/v1/core/entry/{pdb_id}"
    r = sess.get(url)
    data = json.loads(r.text)
    return data["rcsb_entry_container_identifiers"].get("polymer_entity_ids")

# match PDB entity to protein chain
def get_PDB_entity_with_chain(entity_ids,pdb_id,chain):
    for entity in entity_ids:
        url = f"https://data.rcsb.org/rest/v1/core/polymer_entity/{pdb_id}/{entity}"
        r = sess.get(url)
        data = json.loads(r.text)
        # assumption: FoldSeek gives author chain naming, not PDB renamed chain names
        if chain in data["rcsb_polymer_entity_container_identifiers"].get("auth_asym_ids"):
            return entity
        else:
            continue

# get UniProt ID based on the PDB ID and entity (corresponding to the chain reported by FoldSeek)
def get_UniProt_from_PDB_entity(pdb_id,entity):
    url = f"https://data.rcsb.org/rest/v1/core/polymer_entity/{pdb_id}/{entity}"
    r = sess.get(url)
    if not r.ok:
      r.raise_for_status()
    data = json.loads(r.text)
    return data["rcsb_polymer_entity_container_identifiers"].get("uniprot_ids")

In [21]:
# instantiate empty dictionaries
dict_pdb_UniProtID_NCBIid = {}
dict_pdb_PDBidch_UniProtID = {}

In [23]:
pipeline_search_dir = "c_structure_annotation/structure_comp"  
# loop through batches, then through proteins
for index_batch in range(1, 6): 
    batch_dir = os.path.join(pipeline_search_dir, f"batch_{index_batch}")
    protein_files = [f for f in os.listdir(batch_dir) if f.endswith('_foldseek_pdb_filtered.csv')]

    for file_name in protein_files:
        protein = file_name.replace('_foldseek_pdb_filtered.csv', '')
  
        filtered_foldseek = pd.read_csv(os.path.join(batch_dir, f"{protein}_foldseek_pdb_filtered.csv"))
        # check if it has any content
        if not filtered_foldseek.empty:
            # loop over entries
            for i in range(0,len(filtered_foldseek)):
                # extract PDB ID and chain
                pdb_id, chain = get_PDBid_chain_from_target(filtered_foldseek.at[i,"target"])
                # if PDB ID chain combo already in dictionary
                if f"{pdb_id}_{chain}" in dict_pdb_PDBidch_UniProtID.keys():
                    # get UniProt ID previously stored
                    uniprot_ids = dict_pdb_PDBidch_UniProtID.get(f"{pdb_id}_{chain}")
                    if uniprot_ids is not None:
                        for uniprot_id in uniprot_ids:
                            # add NCBI ID to list of matched UniProt IDs
                            if uniprot_id not in dict_pdb_UniProtID_NCBIid.keys():
                                prot_list = list()
                            if uniprot_id in dict_pdb_UniProtID_NCBIid.keys():
                                prot_list = dict_pdb_UniProtID_NCBIid.get(uniprot_id)
                            prot_list.append(protein)
                            dict_pdb_UniProtID_NCBIid[uniprot_id] = list(set(prot_list))
                    
                # if PDB ID chain combo not yet in dictionary
                if f"{pdb_id}_{chain}" not in dict_pdb_PDBidch_UniProtID.keys():
                    # check if PDB ID is obsolete
                    if not check_PDBid_active(pdb_id, obsolete_ids):
                        # if obsolete, instead of storing a UniProt ID we store "Obsolete PDB ID"
                        dict_pdb_PDBidch_UniProtID[f"{pdb_id}_{chain}"] = "Obsolete PDB ID"
                    if check_PDBid_active(pdb_id, obsolete_ids):
                        # fetch UniProt ID(s) - this can be multiple
                        uniprot_ids = get_UniProt_from_PDB_entity(pdb_id,get_PDB_entity_with_chain(get_PDB_entities_from_entry(pdb_id),pdb_id,chain))
                        # add link PDB id chain combo to UniProt IDs
                        dict_pdb_PDBidch_UniProtID[f"{pdb_id}_{chain}"] = uniprot_ids
                        if uniprot_ids is not None:
                        # link UniProt IDs to NCBI protein ids
                            for uniprot_id in uniprot_ids:
                                if uniprot_id not in dict_pdb_UniProtID_NCBIid.keys():
                                    prot_list = list()
                                if uniprot_id in dict_pdb_UniProtID_NCBIid.keys():
                                    prot_list = dict_pdb_UniProtID_NCBIid.get(uniprot_id)
                                prot_list.append(protein)
                                dict_pdb_UniProtID_NCBIid[uniprot_id] = list(set(prot_list))

In [24]:
  # store dictionaries to file
    # dictionary storing link UniProt ID - NCBI ID
with open(os.path.join(pipeline_search_dir, "dict_pdb_UniProtID_NCBIid.csv"), "w", newline="") as fp:
    writer = csv.DictWriter(fp, fieldnames = dict_pdb_UniProtID_NCBIid.keys())
    writer.writeheader()
    writer.writerow(dict_pdb_UniProtID_NCBIid)
    # dictionary storing link PDB entry (ID and chain) - UniProt ID
with open(os.path.join(pipeline_search_dir, "dict_pdb_PDBidch_UniProtID.csv"), "w", newline="") as fp:
    writer = csv.DictWriter(fp, fieldnames = dict_pdb_PDBidch_UniProtID.keys())
    writer.writeheader()
    writer.writerow(dict_pdb_PDBidch_UniProtID)

Step 2 - Obtaining the UniProt function description based on the UniProt IDs
Next, we will extract the function description for all proteins for which we obtained the UniProt IDs from the filtered FoldSeek results. Executed July 9-10 Feb, 2025, UniProt release 2025_01.

In [25]:
# functions to ...

# get protein name, for a single UniProt ID
def get_protein_name_string(uniprot_id):
    # note: unsure if protein_name is a required field, if we get errors, look into this!
    url = f"https://rest.uniprot.org/uniprotkb/search?query={uniprot_id}&fields=protein_name&format=tsv"
    r = sess.get(url)
    r.raise_for_status()
    content = r.text
    names = content.split('\n')[1:-1]
    # if the UniProt entry was marked as obsolete, access its UniParc accession
    if "deleted" in names:
        try:
            return ";".join([str(name) for name in get_uniparc_entry(uniprot_id)])
        except TypeError:
            return "WARNING UniProt ID is obsolete and issue with fetching UniParc information"
    else:
        return ";".join([str(name) for name in names])

# get protein names (and whether the protein is still in UniProt) from a UniParc entry matching the obsolete UniProt ID
def get_uniparc_entry(uniprot_id):
    requestURL = f"https://www.ebi.ac.uk/proteins/api/uniparc/accession/{uniprot_id}"
    r = requests.get(requestURL, headers={ "Accept" : "application/json"})
    if not r.ok:
      r.raise_for_status()
    responseBody = r.text
    data = json.loads(r.text)
    uniparc_entries = []
    uniparc_entries.append("WARNING UniProt ID obsolete, info fetched from UniParc")
    for i in range(0,len(data["dbReference"])):
        for j in data["dbReference"][i].get("property"):
            if j.get("type") == "protein_name":
                uniparc_entries.append(f"{data['dbReference'][i].get('id')} : {j.get('value')} (active UniProt ID: {data['dbReference'][i].get('active')})")
    return uniparc_entries

In [26]:
# list of failed IDs
failed_UniProtID_function = list()
# dict storing functions
dict_UniProtID_function = {}
# get all the UniProt IDs from the two dictionaries:
uniprot_to_search = list(set(list(dict_af50m_UniProtID_NCBIid.keys()) + list(dict_pdb_UniProtID_NCBIid.keys())))

In [ ]:
# search each element, store in dict
for uniprot_id in uniprot_to_search:
    # this loop allows us to rerun upon errors
    if uniprot_id in dict_UniProtID_function.keys():
        continue
    if uniprot_id not in dict_UniProtID_function.keys():
        try:
            function = get_protein_name_string(uniprot_id)
            dict_UniProtID_function[uniprot_id] = function
        except requests.exceptions.ReadTimeout:
            failed_UniProtID_function.append(uniprot_id)
            print(f"Failed to fetch information on UniProt ID {uniprot_id} due to ReadTimeout error. Stored this ID to the list of failed IDs. Try again later.")
        except JSONDecodeError:
            failed_UniProtID_function.append(uniprot_id)
            print(f"Failed to fetch information on UniProt ID {uniprot_id} due to JSON decode error. Stored this ID to the list of failed IDs. Try again later.")
        except requests.exceptions.ConnectionError:
            failed_UniProtID_function.append(uniprot_id)
            print(f"Failed to fetch information on UniProt ID {uniprot_id} due to Connection error. Stored this ID to the list of failed IDs. Try again later.")

In [ ]:
# storing the dictionary to file
with open(os.path.join(pipeline_search_dir, "dict_UniProtID_function.csv"), "w", newline="") as fp:
    # Create a writer object
    writer = csv.DictWriter(fp, fieldnames=dict_UniProtID_function.keys())
    # Write the data rows
    writer.writeheader()
    writer.writerow(dict_UniProtID_function)

In [ ]:
dict_UniProtID_function = {}
failed_UniProtID_function = []

for uniprot_id in uniprot_to_search:
    if uniprot_id in dict_UniProtID_function.keys():
        continue
    try:
        function = get_protein_name_string(uniprot_id)
        dict_UniProtID_function[uniprot_id] = function
    except requests.exceptions.ReadTimeout:
        failed_UniProtID_function.append(uniprot_id)
        print(f"Failed to fetch information on UniProt ID {uniprot_id} due to ReadTimeout error.")
    except JSONDecodeError:
        failed_UniProtID_function.append(uniprot_id)
        print(f"Failed to fetch information on UniProt ID {uniprot_id} due to JSON decode error.")
    except requests.exceptions.ConnectionError:
        failed_UniProtID_function.append(uniprot_id)
        print(f"Failed to fetch information on UniProt ID {uniprot_id} due to Connection error.")

    # Write current state to CSV after each iteration
    with open(os.path.join(pipeline_search_dir, "dict_UniProtID_function.csv"), "w", newline="") as fp:
        writer = csv.writer(fp)
        writer.writerow(["UniProtID", "Function"])
        for key, value in dict_UniProtID_function.items():
            writer.writerow([key, value])

print("Final dictionary saved.")


In [29]:
#to run if above cell was not completed in one session
import csv
import os
import requests
from json import JSONDecodeError

csv.field_size_limit(10 * 1024 * 1024)  # Increase limit to 10MB

pipeline_search_dir = "c_structure_annotation/structure_comp"  
csv_path = os.path.join(pipeline_search_dir, "dict_UniProtID_function.csv")

dict_UniProtID_function = {}
failed_UniProtID_function = []

# Load existing data from CSV if it exists
if os.path.exists(csv_path):
    with open(csv_path, newline='') as fp:
        reader = csv.reader(fp)
        next(reader)  # Skip header
        for row in reader:
            if row:
                dict_UniProtID_function[row[0]] = row[1]

for uniprot_id in uniprot_to_search:
    if uniprot_id in dict_UniProtID_function:
        continue  # Skip if already processed

    try:
        function = get_protein_name_string(uniprot_id)
        dict_UniProtID_function[uniprot_id] = function
    except requests.exceptions.ReadTimeout:
        failed_UniProtID_function.append(uniprot_id)
        print(f"Failed to fetch {uniprot_id}: ReadTimeout error.")
    except JSONDecodeError:
        failed_UniProtID_function.append(uniprot_id)
        print(f"Failed to fetch {uniprot_id}: JSON decode error.")
    except requests.exceptions.ConnectionError:
        failed_UniProtID_function.append(uniprot_id)
        print(f"Failed to fetch {uniprot_id}: Connection error.")

    # Incrementally save to CSV after each processed item
    with open(csv_path, "w", newline="") as fp:
        writer = csv.writer(fp)
        writer.writerow(["UniProtID", "Function"])
        for key, value in dict_UniProtID_function.items():
            writer.writerow([key, value])

print("Dictionary updated and saved.")


Dictionary updated and saved.


 Step 3: Writing output files with function annotations 

In [30]:
# instantiate lists
    # one with all the UniProt IDs for which the function description contains 'acetyltransferase' or 'GNAT'
uniprot_ids_to_annotate = list()
    # one with all the NCBI IDs linked to these UniProt IDs 
    # (i.e., proteins for which at least one of the filtered FoldSeek hits was annotated as 'acetyltransferase' or 'GNAT')
ncbi_ids_to_annotate = list()

In [31]:
# loop over function dict
for key, value in dict_UniProtID_function.items():
    # get UniProt IDs related to function of interest
    uniprot_ids_to_annotate.append(key)


# loop over list of UniProt IDs with function of interest
for uniprot_id in list(set(uniprot_ids_to_annotate)):
    # get NCBI ids for proteins for which we want to annotate the FoldSeek results fully
    ncbi_ids_to_annotate.append(dict_pdb_UniProtID_NCBIid.get(uniprot_id))
    ncbi_ids_to_annotate.append(dict_af50m_UniProtID_NCBIid.get(uniprot_id))

ncbi_ids_to_annotate = [i for i in list(set(collapse(ncbi_ids_to_annotate))) if i is not None]  

In [32]:
 # store this list of NCBI ids to file
with open(os.path.join(pipeline_search_dir, "list_ncbiid_possibleact.txt"), "w+") as file:
    for ncbi_id in ncbi_ids_to_annotate:
        file.write(f"{ncbi_id}\n")
file.close()

In [15]:
pipeline_search_dir = "c_structure_annotation/structure_comp"  
csv_path = os.path.join(pipeline_search_dir, "dict_pdb_PDBidch_UniProtID.csv")

with open(csv_path, 'r') as f:
    lines = f.readlines()

keys = lines[0].strip().split(',')
values = lines[1].strip().split(',')
dict_pdb_PDBidch_UniProtID = dict(zip(keys, values))


In [17]:
pipeline_search_dir = "c_structure_annotation/structure_comp"  
csv_path = os.path.join(pipeline_search_dir, "dict_UniProtID_function.csv")
dict_UniProtID_function = pd.read_csv(csv_path).set_index('UniProtID')['Function'].to_dict()


In [19]:
with open(os.path.join(pipeline_search_dir, "list_ncbiid_possibleact.txt"), "r") as file:
    ncbi_ids_to_annotate = [line.strip() for line in file.readlines()]

In [25]:
for index_batch in range(1, 6): 
    batch_dir = os.path.join(pipeline_search_dir, f"batch_{index_batch}")
    protein_files = [f for f in os.listdir(batch_dir) if f.endswith('_foldseek_af50m_filtered.csv')]

    for file_name in protein_files:
        protein = file_name.replace('_foldseek_af50m_filtered.csv', '')

        data_af50m_filtered = pd.read_csv(os.path.join(batch_dir, f"{protein}_foldseek_af50m_filtered.csv")) 
        # Check if it has any content
        if len(data_af50m_filtered) != 0:
            for i in range(len(data_af50m_filtered)):
                if protein in ncbi_ids_to_annotate:
                    data_af50m_filtered.at[i, "protein_descr"] = dict_UniProtID_function.get(get_UniProtID_from_target(data_af50m_filtered.at[i, "target"]))
            # Store results
            data_af50m_filtered.to_csv(os.path.join(batch_dir, f"{protein}_foldseek_af50m_processed.csv"), index=False)     

        # PDB hits
        data_pdb_filtered = pd.read_csv(os.path.join(batch_dir, f"{protein}_foldseek_pdb_filtered.csv")) 
        if len(data_pdb_filtered) != 0:
            for i in range(len(data_pdb_filtered)):
                pdb_id, chain = get_PDBid_chain_from_target(data_pdb_filtered.at[i, "target"])
                # Get UniProt ID or obsolete from dictionary
                uniprot_ids = dict_pdb_PDBidch_UniProtID.get(f"{pdb_id}_{chain}")
                if uniprot_ids == "Obsolete PDB ID":
                    data_pdb_filtered.at[i, "protein_descr"] = "WARNING This PDB ID is obsolete, hence, no information could be fetched"
                elif uniprot_ids is None:
                    data_pdb_filtered.at[i, "protein_descr"] = "WARNING No UniProt ID was linked to this PDB entry, hence, no information could be fetched"
                else:
                    string = ""
                    for uniprot_id in uniprot_ids.split(';'):  # Assuming multiple UniProt IDs are separated by semicolons
                        string += dict_UniProtID_function.get(uniprot_id, 'Unknown Function') + f" [UniProt ID: {uniprot_id}] "
                    data_pdb_filtered.at[i, "protein_descr"] = string.strip()
            # Store results
            data_pdb_filtered.to_csv(os.path.join(batch_dir, f"{protein}_foldseek_pdb_processed.csv"), index=False)


we have the alphafold and PDB results
now moving on to phold

In [9]:
# cleaning function to remove extra details in query and target col so easier matching with known_annot_phrogs 
def clean_foldseek_ids(query, target):
    clean_query = re.sub(r'_relaxed$', '', query)
    clean_target = re.sub(r'^envhog_', '', target)
    clean_target = re.sub(r'protein', '', clean_target)
    clean_target = re.sub(r'\.\w+$', '', clean_target) 
    
    return clean_query, clean_target

results = []

In [19]:
pipeline_search_dir = "c_structure_annotation/structure_comp"  
csv_path = os.path.join(pipeline_search_dir, "known_annot_phrogs.tsv")
known_annot = pd.read_csv(csv_path, sep='\t')

# Merge "query" and "target" columns into a new column "query:target"
known_annot["query:target"] = known_annot["query"].astype(str) + ":" + known_annot["target"].astype(str)

# Keep only the new column along with "annot" and "category"
known_annot = known_annot[["query:target", "annot", "category"]]

# Save the updated file
updated_file_path = "phrogs_annot.tsv"  
known_annot.to_csv(updated_file_path, sep='\t', index=False)



In [ ]:
def clean_foldseek_ids(query, target):
    clean_query = re.sub(r'_relaxed$', '', query)
    clean_target = re.sub(r'^envhog_', '', target)
    clean_target = re.sub(r'protein', '', clean_target)
    clean_target = re.sub(r'\.\w+$', '', clean_target)
    return clean_query, clean_target

def extract_phrog_number(target):
    match = re.search(r'phrog_(\d+)', target)
    return match.group(1) if match else None


pipeline_search_dir = "c_structure_annotation/structure_comp"  
csv_path = os.path.join(pipeline_search_dir, "known_annot_phrogs.tsv")
known_annot = pd.read_csv(csv_path, sep='\t')
known_annot['phrog_number'] = known_annot['query'].apply(lambda x: re.search(r'phrog_(\d+)', x).group(1) if re.search(r'phrog_(\d+)', x) else None)


for index_batch in range(1, 5):
    batch_dir = os.path.join(pipeline_search_dir, f"batch_{index_batch}")
    
    protein_files = [f for f in os.listdir(batch_dir) if f.endswith('_foldseek_phold_filtered.csv')]

    for file_name in protein_files:
        protein = file_name.replace('_foldseek_phold_filtered.csv', '')
        data_phold_filtered = pd.read_csv(os.path.join(batch_dir, file_name))
        data_phold_filtered['phrog_number'] = data_phold_filtered['target'].apply(extract_phrog_number)

        results = []
        for _, row in data_phold_filtered.iterrows():
            phrog_num = row['phrog_number']
            match = known_annot[known_annot['phrog_number'] == phrog_num]
            annotation = match['annot'].values[0] if not match.empty else 'Uncharacterized Protein'
            results.append(annotation)

        data_phold_filtered['annotation'] = results
        data_phold_filtered.to_csv(os.path.join(batch_dir, f"{protein}_foldseek_phold_processed.csv"), index=False)


